# Digital Methods - Data Analysis
_____

## Table of Content

1. [Libraries](#libraries)
2. [Data Preprocessing](#data-preprocessing)

_____

## Libraries

All libraries which are needed to execute the code are listed here. Install the packages by using the `requirements.txt` file. 

The documentation can be found in the [README.md](README.md) file.

In [45]:
# import packages
import pandas as pd 
import os

# import functions
from preprocessing_function import *

## Data Preprocessing

In [46]:
# set working directory
os.chdir('')
df = pd.read_csv('comments.csv', index_col=0)

#set pandas option to show more text
pd.set_option('display.max_colwidth', 2000)

In [47]:
# process data with using functions from functions.py
processed_df = (
    df.pipe(remove_users, 'text')
      .pipe(lowercase_text, 'text')
      .pipe(remove_numbers, 'text')
      .pipe(remove_whitespace, 'text')
      .pipe(remove_stopwords, 'text')
      .pipe(remove_punctuation, 'text')
      .pipe(remove_whitespace, 'text')
)

In [63]:
processed_df['text'] = processed_df['text'].astype('string')
df.dtypes

video_id        object
published_at    object
like_count       int64
text            string
author          object
dtype: object

In [66]:
# use stemming to reduce words to their root words
processed_df = stem_words(processed_df, 'text')

In [68]:
# use lemmatization to reduce words to their root form
processed_df = lemmatize_words(processed_df, 'text')

In [70]:
processed_df.to_csv("processed_comments.csv")